<a href="https://colab.research.google.com/github/LosGuys/frac1/blob/main/frac1_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import cv2
from datetime import datetime
from numba import jit, cuda
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor

@cuda.jit
def mandelbrot_gpu(xmin, xmax, ymin, ymax, width, height, max_iter, output):
    # Calculate the position in the grid
    x, y = cuda.grid(2)
    if x < width and y < height:
        re = xmin + (x / width) * (xmax - xmin)
        im = ymin + (y / height) * (ymax - ymin)
        c = complex(re, im)
        z = c
        for n in range(max_iter):
            if abs(z) > 2.0:
                break
            z = z * z + c
        color = 255 - int(n * 255 / max_iter)
        r = (color * 9) % 256
        g = (color * 7) % 256
        b = (color * 5) % 256
        output[y, x, 0] = b
        output[y, x, 1] = g
        output[y, x, 2] = r

def generate_frame_gpu(xmin, xmax, ymin, ymax, width, height, max_iter):
    # Allocate memory on GPU and CPU
    output = np.zeros((height, width, 3), dtype=np.uint8)
    d_output = cuda.to_device(output)

    # Define thread and block sizes
    threads_per_block = (16, 16)
    blocks_per_grid_x = (width + threads_per_block[0] - 1) // threads_per_block[0]
    blocks_per_grid_y = (height + threads_per_block[1] - 1) // threads_per_block[1]
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    # Launch the kernel
    mandelbrot_gpu[blocks_per_grid, threads_per_block](xmin, xmax, ymin, ymax, width, height, max_iter, d_output)

    # Copy the result back to host memory
    output = d_output.copy_to_host()
    return output

def create_video(duration, fps, resolution, max_iter, colormap, zoom_factor=0.95, center=(-0.75, 0), adjust_iter=True):
    """
    Create a zooming Mandelbrot video using GPU acceleration.
    """
    width, height = resolution
    frames = int(duration * fps)
    xmin, xmax, ymin, ymax = -2.5, 1.5, -2, 2

    # Set the initial bounds centered on the provided center
    x_center, y_center = center
    x_range, y_range = xmax - xmin, ymax - ymin
    xmin, xmax = x_center - x_range / 2, x_center + x_range / 2
    ymin, ymax = y_center - y_range / 2, y_center + y_range / 2

    timestamp = datetime.now().strftime("%y.%m.%d_%H.%M.%S")
    filename = f"mandelbrot_zoom_{timestamp}.mp4"
    video = cv2.VideoWriter(
        filename,
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (width, height)
    )

    # Generate frames using GPU
    for frame in range(frames):
        print(f"Rendering frame {frame + 1}/{frames}...")

        # Dynamically increase max iterations to maintain detail
        if adjust_iter:
            max_iter = int(max_iter * (1 + frame / frames))

        image = generate_frame_gpu(xmin, xmax, ymin, ymax, width, height, max_iter)
        video.write(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

        # Zoom in by shrinking bounds
        x_range, y_range = x_range * zoom_factor, y_range * zoom_factor
        xmin, xmax = x_center - x_range / 2, x_center + x_range / 2
        ymin, ymax = y_center - y_range / 2, y_center + y_range / 2

    video.release()
    print(f"Video saved as {filename}!")

if __name__ == "__main__":
    # Configuration
    duration = 90  # seconds
    fps = 30       # frames per second
    resolution = (1920, 1080)  # width, height (Full HD)
#    resolution = (480, 270)  # width, height (Speed run)
    max_iter = 500  # initial complexity of the Mandelbrot set
    colormap = "twilight"  # vibrant color mapping (not used in this GPU approach)
    zoom_factor = 0.99  # zooming effect per frame
    center = (-0.7436395, 0.131825904205330)  # interesting structured region
    adjust_iter = False  # dynamically increase max_iter for better zoom detail

    create_video(duration, fps, resolution, max_iter, colormap, zoom_factor, center, adjust_iter)

#    subprocess.Popen('explorer '+ os.getcwd())


Rendering frame 1/2700...
Rendering frame 2/2700...
Rendering frame 3/2700...
Rendering frame 4/2700...
Rendering frame 5/2700...
Rendering frame 6/2700...
Rendering frame 7/2700...
Rendering frame 8/2700...
Rendering frame 9/2700...
Rendering frame 10/2700...
Rendering frame 11/2700...
Rendering frame 12/2700...
Rendering frame 13/2700...
Rendering frame 14/2700...
Rendering frame 15/2700...
Rendering frame 16/2700...
Rendering frame 17/2700...
Rendering frame 18/2700...
Rendering frame 19/2700...
Rendering frame 20/2700...
Rendering frame 21/2700...
Rendering frame 22/2700...
Rendering frame 23/2700...
Rendering frame 24/2700...
Rendering frame 25/2700...
Rendering frame 26/2700...
Rendering frame 27/2700...
Rendering frame 28/2700...
Rendering frame 29/2700...
Rendering frame 30/2700...
Rendering frame 31/2700...
Rendering frame 32/2700...
Rendering frame 33/2700...
Rendering frame 34/2700...
Rendering frame 35/2700...
Rendering frame 36/2700...
Rendering frame 37/2700...
Rendering 